In [1]:
import numpy as np
from random import choice, seed
from math import isnan
import csv
import pyproj

In [46]:
def ll2ps(lat, lon):
    # Define the projection parameters
    proj_params = {
        'proj': 'stere',
        'lat_0': 90, # suitable for the southern hemisphere
        'lon_0': 0,
        'lat_ts': -71,
        'a': 6378137,
        'b': 6356752.3
    }

    # Create the projection object
    proj = pyproj.Proj(proj_params)

    # Convert latitude and longitude to polar stereographic coordinates
    x, y = proj(lon, lat)

    return x, y

In [75]:
def scarloc(featurename, *varargin):
    
    # Check if featurename is a list or a single string
    if isinstance(featurename, str):
        featurename = [featurename]

    # Handle optional arguments
    OfferHelp = False
    pscoords = False
    kmout = False

    if len(varargin) > 0:
        if isinstance(varargin[0], bool):
            OfferHelp = varargin[0]
        if len(varargin) >= 1:
            if 'xy' in varargin[0]:
                pscoords = True
                if len(varargin) >= 2 and 'km' in varargin[1]:
                    kmout = True

    # Load data from CSV file
    lat, lon, names = load_csv_data('SCAR_CGA_PLACE_NAMES.csv')

    featurelat = np.full(len(featurename), np.nan)
    featurelon = np.full(len(featurename), np.nan)

    # Look for each feature name
    for k in range(len(featurelat)):
        x, NearbyNames = strlookup(featurename[k], names)
        if x is None and OfferHelp:
            fmsg = [
                f'"{featurename[k]}" not found.',
                f'Are you sure that "{featurename[k]}" exists in Antarctica?',
                'Did a cat walk across your keyboard?',
                'This is the real reason one shouldn''t text and drive. Check your spelling and try again.',
                'Now you''re just making things up.',
                f'SCAR has identified more than 25,000 features in Antarctica, but "{featurename[k]}" is not one of them.',
                f'Can''t find "{featurename[k]}".',
                f'"{featurename[k]}" may exist somewhere in the world, but you won''t find it in Antarctica.',
                f'It is possible that Robert F. Scott named something in Antarctica "{featurename[k]}", but if he did there are no records of it.',
                f'You must be thinking of {featurename[k]}, Kansas, because {featurename[k]}, Antarctica does not exist.',
                f'Sure, they used to just call it {featurename[k]}, but not anymore, what with political correctness and all.',
                f'"{featurename[k]}" is an interesting combination of letters, but I don''t think it''s any place in Antarctica.',
                f'The great Wayne Cochran once sang, "Where oh where can my {featurename[k]} be?" Because it''s not in Antarctica.',
                f'I''m pretty sure it is in violation of the Antarctic Treaty to refer to any place as "{featurename[k]}".',
                f'"{featurename[k]}" does not match any entries in the SCAR database.',
                f'Science is all about formality, so the bigwigs will surely look down their noses at such colloquial jargon as "{featurename[k]}".',
                f'My doctor said I need to get my {featurename[k]} removed.',
                'Frostbitten Antarctic researcher mistypes again.',
                'This may be an issue of American English versus British English.',
                f'Antarctica''s a strange place, but it''s not science fiction. Verify that "{featurename[k]}" actually exists.',
                f'What''s in a name? I''ll tell you what''s in a name: That which you call "{featurename[k]}" by any other name may actually exist in Antarctica.',
                f'Did John Carpenter tell you''ll find "{featurename[k]}" in Antarctica?',
                f'You know, some folks say glaciology is a shrinking field, but I say things are just heating up. In other news, "{featurename[k]}" does not exist.',
                f'You''re a glaciologist? Isn''t that a slow-moving field? Also, I have to tell you, I can''t seem to find any record of "{featurename[k]}".',
                f'Amazing glaciology, how sweet the sound... "{featurename[k]}" once was lost, and still has not been found.'
            ]

            np.random.shuffle(fmsg)
            print(fmsg[0])
            if NearbyNames:
                print('Here are the best matches I can find:')
                print(NearbyNames)
            else:
                print('Try typing "load scarnames" to explore the available list of features.')
            return

        if x is not None:
            featurelat[k] = lat[x]
            featurelon[k] = lon[x]
    
    # Convert to polar stereographic coordinates
    if pscoords:
        featurelat, featurelon = ll2ps(featurelat, featurelon)

    # Convert to polar stereographic kilometers
    if kmout:
        featurelon = featurelon / 1000
        featurelat = featurelat / 1000

    # Returning only latitude or only x would not make any sense,
    # so if no outputs are requested, or if only one output is requested,
    # return as a lat column and lon column or [x y]
    if len(featurename) == 1:
        varargout = np.column_stack((featurelat, featurelon))
        return varargout[0]
    else:
        varargout = [featurelat, featurelon]
        return varargout


In [76]:
def load_csv_data(filename):
    lat = []
    lon = []
    names = []

    with open(filename, 'r', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip header row

        for row in csvreader:
            try:
                lat_val = float(row[5])
                lon_val = float(row[6])
                lat.append(lat_val)
                lon.append(lon_val)
                names.append(row[1])
            except ValueError:
                continue

    return np.array(lat), np.array(lon), np.array(names)

In [77]:
def strlookup(featurename, names):
    featurename = featurename.lower()  # Convert featurename to lowercase for case-insensitive matching

    indices = [i for i, name in enumerate(names) if featurename in name.lower()]

    if len(indices) > 0:
        x = indices[0]  # Choose the first matching index
        NearbyNames = [names[i] for i in indices]  # Get all matching names
    else:
        x = None
        NearbyNames = []

    return x, NearbyNames

In [78]:
def handle_missing_feature(featurename, NearbyNames):
    fmsg = [
        f'"{featurename}" not found.',
        f'Are you sure that "{featurename}" exists in Antarctica?',
        'Did a cat walk across your keyboard?',
        'This is the real reason one shouldn\'t text and drive. Check your spelling and try again.',
        'Now you\'re just making things up.',
        f'SCAR has identified more than 25,000 features in Antarctica, but "{featurename}" is not one of them.',
        f'Can\'t find "{featurename}".',
        f'"{featurename}" may exist somewhere in the world, but you won\'t find it in Antarctica.',
        f'It is possible that Robert F. Scott named something in Antarctica "{featurename}", but if he did there are no records of it.',
        f'You must be thinking of {featurename}, Kansas, because {featurename}, Antarctica does not exist.',
        f'Sure, they used to just call it {featurename}, but not anymore, what with political correctness and all.',
        f'"{featurename}" is an interesting combination of letters, but I don\'t think it\'s any place in Antarctica.',
        f'The great Wayne Cochran once sang, "Where oh where can my {featurename} be?" Because it\'s not in Antarctica.',
        f'I\'m pretty sure it is in violation of the Antarctic Treaty to refer to any place as "{featurename}".',
        f'"{featurename}" does not match any entries in the SCAR database.',
        f'Science is all about formality, so the bigwigs will surely look down their noses at such colloquial jargon as "{featurename}".',
        f'My doctor said I need to get my {featurename} removed.',
        'Frostbitten Antarctic researcher mistypes again.',
        'This may be an issue of American English versus British English.',
        f'Antarctica\'s a strange place, but it\'s not science fiction. Verify that "{featurename}" actually exists.',
        f'What\'s in a name? I\'ll tell you what\'s in a name: That which you call "{featurename}" by any other name may actually exist in Antarctica.',
        f'Did John Carpenter tell you\'ll find "{featurename}" in Antarctica?',
        f'You know, some folks say glaciology is a shrinking field, but I say things are just heating up. In other news, "{featurename}" does not exist.',
        f'You\'re a glaciologist? Isn\'t that a slow-moving field? Also, I have to tell you, I can\'t seem to find any record of "{featurename}".',
        f'Amazing glaciology, how sweet the sound... "{featurename}" once was lost, and still has not been found.'
    ]

    rngstart = seed()  # get initial rng setting before changing it temporarily.
    random_msg = choice(fmsg)
    print(random_msg)
    seed(rngstart)  # returns to original rng settings.

    if NearbyNames:
        print('Here are the best matches I can find:')
        print(NearbyNames)
    else:
        print('Try typing "load scarnames" to explore the available list of features.')

    return np.nan, np.nan

In [86]:
[lat,lon] = scarloc('McMurdo Station')
print(lat)
print(lon)

-77.8478
166.6683


In [81]:
latlon = scarloc('mcmurdo station')
print(latlon)

[-77.8478 166.6683]


In [82]:
places = ['Byrd Camp','Casey Station','Pine Island Glacier']
scarloc(places)

[array([-80.0833, -66.2833, -75.    ]),
 array([-119.5333,  110.5194, -101.    ])]

In [84]:
scarloc(['Byrd Camp','Casey Station','Pine Island Glacier'],'xy')

[array([ -939729.52149631,  2447051.54179615, -1608674.17796745]),
 array([-532394.61214777, -915859.58138777, -312694.58401518])]

In [85]:
scarloc(['Byrd Camp','Casey Station','Pine Island Glacier'],'xy','km')

[array([ -939.7295215 ,  2447.0515418 , -1608.67417797]),
 array([-532.39461215, -915.85958139, -312.69458402])]